In [3]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
ash_excitation = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity


#Hamiltonian 
symbols = ["H","H","H","H"]
r_bohr = 5.0*1.8897259886
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")

#print('The original hamiltonian is', H)
electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals,fermionic=True)
print('Singles are',singles)
print('Doubles are',doubles)
hf_state = qchem.hf_state(electrons, qubits)
print(f"Total number of excitations = {len(singles) + len(doubles)}")

#Calculation of HF state
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit(hf_state, electrons, qubits, H):
    # Prepare the Hartree-Fock state
    print('Updated hf_state is', hf_state)
    qml.BasisState(hf_state, wires=range(qubits))

    return qml.expval(H)
print('HF state is', circuit(hf_state, electrons, qubits, H))

#Putting all the functions here
#1.1091571486954503

Singles are [FermiWord({(0, 0): '+', (1, 4): '-'}), FermiWord({(0, 0): '+', (1, 6): '-'}), FermiWord({(0, 1): '+', (1, 5): '-'}), FermiWord({(0, 1): '+', (1, 7): '-'}), FermiWord({(0, 2): '+', (1, 4): '-'}), FermiWord({(0, 2): '+', (1, 6): '-'}), FermiWord({(0, 3): '+', (1, 5): '-'}), FermiWord({(0, 3): '+', (1, 7): '-'})]
Doubles are [FermiWord({(0, 0): '+', (1, 1): '+', (2, 4): '-', (3, 5): '-'}), FermiWord({(0, 0): '+', (1, 1): '+', (2, 4): '-', (3, 7): '-'}), FermiWord({(0, 0): '+', (1, 1): '+', (2, 5): '-', (3, 6): '-'}), FermiWord({(0, 0): '+', (1, 1): '+', (2, 6): '-', (3, 7): '-'}), FermiWord({(0, 0): '+', (1, 2): '+', (2, 4): '-', (3, 6): '-'}), FermiWord({(0, 0): '+', (1, 3): '+', (2, 4): '-', (3, 5): '-'}), FermiWord({(0, 0): '+', (1, 3): '+', (2, 4): '-', (3, 7): '-'}), FermiWord({(0, 0): '+', (1, 3): '+', (2, 5): '-', (3, 6): '-'}), FermiWord({(0, 0): '+', (1, 3): '+', (2, 6): '-', (3, 7): '-'}), FermiWord({(0, 1): '+', (1, 2): '+', (2, 4): '-', (3, 5): '-'}), FermiWord({(

In [5]:
# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_0(H,w, k):
    qml.BasisState(k, wires=range(qubits))
    res = qml.commutator(H, w)
    #res = 2*(qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Commutator calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def commutator_1(H,w, k):
    qml.StatePrep(k, wires=range(qubits))
    res = qml.commutator(H, w)
    #res = 2*(qml.prod(H, w)) - (qml.prod(w, H))
    return qml.expval(res)

# Energy calculation 
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def ash(params, ash_excitation, hf_state, qubits, H):
    qml.BasisState(hf_state, wires=range(qubits))
    for i, excitation in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc going in is', ash_excitation[i])
            qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            #qml.SingleExcitation(params[i], wires=ash_excitation[i])
    return qml.expval(H)

# Calculation of New state 
#print('Now we are going to calculate the new_state')
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def new_state(hf_state, ash_excitation, params, fparams):
    qml.BasisState(hf_state, wires=range(qubits))
    print('All params in state prep are', fparams)
    for i, excitations in enumerate(ash_excitation):
        if len(ash_excitation[i]) == 4:
            print('Exc. dealing right now is', ash_excitation[i])
            print('The params that are going in', fparams[i])
            #print('The wires1 are', ash_excitation[i][2:][::-1])
            qml.DoubleExcitation(fparams[i], wires = ash_excitation[i])
            #qml.FermionicDoubleExcitation(weight=params[i], wires1=ash_excitation[i][2:][::-1], wires2=ash_excitation[i][:2][::-1])
        elif len(ash_excitation[i]) == 2:
            print('Single Exc. dealing right now is', ash_excitation[i])
            print('Single exc params that are going in', fparams[i])
            #qml.FermionicSingleExcitation(weight=params[i], wires=ash_excitation[i])
            qml.SingleExcitation(fparams[i], wires=ash_excitation[i])

    return qml.state()

In [7]:

electrons = 4
orbitals = 8
singles, doubles = qml.qchem.excitations(electrons, orbitals, fermionic=True)
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)

print('The Hamiltonian is ', H)
adit = 6
fparams = []
excitations= []
operator_pool = (singles) + (doubles)
states = [hf_state]


for j in range(adit):
    print('The adapt iteration now is', j)
    max_value = float('-inf')
    max_operator = None
    # Start with the most recent state (last state in the list)
    k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state

    for i in operator_pool:
        print('The current excitation operator is', i)
        w = qml.fermi.jordan_wigner(i)
        if np.array_equal(k, hf_state):
            print('Print, if this is activated - HF state')
            current_value = abs(commutator_0(H, w, k))
        else:
            current_value = abs(commutator_1(H, w, k))
        print(f'The expectation value of {i} is', current_value)

        if current_value > max_value:
            max_value = current_value
            max_operator = i

    print(f"The highest operator value is {max_value} for operator {max_operator}")

    # Convert operator to excitations and append to ash_excitation
    indices_str = re.findall(r'\d+', str(max_operator))
    excitations = [int(index) for index in indices_str]
    print('Highest gradient excitation is', excitations)
    ash_excitation.append(excitations)
    print('The current status of ash_excitation is', ash_excitation)
    print('Moving towards parameters')
    params = np.zeros(len(ash_excitation), requires_grad=True)
    print('The length of parameters is', len(params))
    
    # Cost function definition
    cost_fn = qml.QNode(ash, dev, interface="autograd", diff_method="adjoint")
    print('Going to do energy calculation')
    for n in range(150):
        print(f'Each step, the iteration is {n} and the parameter is {params}')
        params, energy = optimizer.step_and_cost(cost_fn, params, ash_excitation=ash_excitation, hf_state=hf_state, qubits=qubits, H=H)
        if n % 5 == 0:
            print(f"step = {n}, E = {energy:.8f} Ha")
    fparams.append(params)
    print('Updated params are', fparams)
    print('Updated excitation are', ash_excitation)
    
    # New state generation
    ostate = new_state(hf_state, ash_excitation, params, fparams)
    print(qml.draw(new_state, level="device", max_length=100)(hf_state,ash_excitation,params,fparams))
    print('The updated state is', ostate)
    # Append the new state to the states list
    states.append(ostate)  


The Hamiltonian is  -1.1091571486954508 * I(0) + 0.04196325905665746 * Z(0) + 6.339400653843563e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.339400653843563e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04183466728352388 * Z(2) + 0.015433496464673638 * (Z(0) @ Z(2)) + 6.344558587066803e-05 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 6.344558587066803e-05 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.0043780384020612705 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + -0.0043780384020612705 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 0.04170505593332367 * Z(4) + 0.019843364998430132 * (Z(0) @ Z(4)) + 0.041576261632177314 * Z(6) + 0.022112211956182883 * (Z(0) @ Z(6)) + 0.04196325905665746 * Z(1) + 0.0627720043383223 * (Z(0) @ Z(1)) + 0.0025025640473308832 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.0025025640473308832 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.04731295041968314 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04731295041968314 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04731295041968314 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.0

/Users/cwq/CodingwithQiskit/miniconda3/envs/cwq/lib/python3.12/site-packages/autoray/autoray.py:1127: ComplexWarning: Casting complex values to real discards the imaginary part
  return x.astype(dtype, **kwargs)


The expectation value of a⁺(0) a(4) is 7.885701387105523e-06
The current excitation operator is a⁺(0) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(0) a(6) is 0.0
The current excitation operator is a⁺(1) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(5) is 7.885701387110727e-06
The current excitation operator is a⁺(1) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(1) a(7) is 0.0
The current excitation operator is a⁺(2) a(4)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(4) is 0.0
The current excitation operator is a⁺(2) a(6)
Print, if this is activated - HF state
The expectation value of a⁺(2) a(6) is 7.806876035386552e-06
The current excitation operator is a⁺(3) a(5)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(5) is 0.0
The current excitation operator is a⁺(3) a(7)
Print, if this is activated - HF state
The expectation value of a⁺(3) a(7) is 7.806876035

IndexError: list index out of range

In [8]:
print(fparams)

[tensor([-0.70744893, -0.70744893], requires_grad=True)]
